In [10]:
# Add the path to the 'my_llm' module
import sys
sys.path.append('../src')

from my_llm.my_utils import load_all_jsonl_files
from my_llm.my_utils import load_normalized_jsonl
from my_llm.my_utils import load_txt_files_to_df
from my_llm.my_utils import split_train_test
from my_llm.gpt_client import load_model_config
from my_llm.gpt_client import create_pipeline

from datasets import Dataset
from sklearn.model_selection import train_test_split
import re
import glob
import os
import pandas as pd
from transformers import pipeline



In [11]:
# Load configuration in a Python file (e.g., main.py or a notebook cell)
import yaml

with open("/Users/blairjdaniel/AI-Assistant-Springs/config/model_config.yaml", "r") as file:
    config = yaml.safe_load(file)

print(config)

{'split': {'test_size': 0.2, 'random_state': 42}, 'pipeline': {'task': 'text-generation', 'model_name': 'EleutherAI/gpt-neo-1.3B'}, 'model': {'name': 'EleutherAI/gpt-neo-1.3B', 'training': {'epochs': 3, 'batch_size': 32, 'learning_rate': '2e-5', 'max_seq_length': 128}}, 'tokenizer': {'name': 'EleutherAI/gpt-neo-1.3B', 'do_lower_case': True, 'truncation': True, 'padding': 'max_length'}, 'huggingface': {'cache_dir': '~/.cache/huggingface', 'use_auth_token': False}, 'logging': {'level': 'INFO', 'log_file': 'logs/training.log'}}


In [12]:
def load_model_config(yaml_path: str) -> dict:
    with open(yaml_path, "r") as file:
        return yaml.safe_load(file)

In [14]:
# Load the config
with open("/Users/blairjdaniel/AI-Assistant-Springs/config/model_config.yaml", "r") as f:
    model_config = yaml.safe_load(f)

pipe = pipeline(
    model_config["pipeline"]["task"],
    model=model_config["pipeline"]["model_name"]
)

Device set to use mps:0


RuntimeError: MPS backend out of memory (MPS allocated: 6.76 GB, other allocations: 628.00 KB, max allowed: 6.77 GB). Tried to allocate 392.63 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:
# # Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")

In [15]:
data_dir = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs"
combined_data = load_all_jsonl_files(data_dir)
print("Combined dataset count:", len(combined_data))
# Normalize the JSON data
# Convert the list to a pandas DataFrame before normalizing
normalized_combined_data = pd.json_normalize(pd.DataFrame(combined_data).to_dict(orient='records'))

Combined dataset count: 716


In [ ]:
# brandbook = pd.read_json('/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/brandbook.jsonl', lines=True)
# normalized_brandbook = pd.json_normalize(pd.DataFrame(brandbook).to_dict(orient='records'))

In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_contact.jsonl"
# normalized_email_contact = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", normalized_email_contact.shape)

Normalized DataFrame shape: (68, 8)


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_inquiries.jsonl"
# normalized_email_inquiries = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", normalized_email_inquiries.shape)


Normalized DataFrame shape: (68, 8)


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_pricelist.jsonl"
# normalized_email_pricelist = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", normalized_email_pricelist.shape)

Normalized DataFrame shape: (391, 8)


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/email_waitlist.jsonl"
# normalized_email_waitlist = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", normalized_email_waitlist.shape)

Normalized DataFrame shape: (25, 8)


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/instagram_enriched.jsonl"
# normalized_instagram = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", normalized_instagram.shape)

Normalized DataFrame shape: (28, 7)


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/training_data.jsonl"
# normalized_training_data = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", normalized_training_data.shape)

Normalized DataFrame shape: (43, 2)


In [ ]:
# file_path = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs/website_enriched.jsonl"
# normalized_website = load_normalized_jsonl(file_path)
# print("Normalized DataFrame shape:", normalized_website.shape)

Normalized DataFrame shape: (78, 6)


In [16]:
txt_directory = "/Users/blairjdaniel/AI-Assistant-Springs/data/outputs"
df_txt = load_txt_files_to_df(txt_directory)
print("Loaded", len(df_txt), "text files")
print(df_txt.head())

Loaded 3 text files
              filename                                            content
0    website_clean.txt  {"text": "Welcome to Springs RV Resort, where ...
1  brandbook_clean.txt   Springs RV Resort Brand Book  \n 1. Company O...
2  documents_clean.txt  Event Checklist BEFORE EVENT Monday Meeting (w...


In [17]:
normalized_combined_data = normalized_combined_data.fillna("")

In [ ]:
data = normalized_combined_data.copy()
data.head()

In [ ]:
# Assuming 'data' and 'df_text' are your DataFrames
combined_df = pd.concat([data, df_txt], axis=0, ignore_index=True, sort=False)
combined_df = combined_df.fillna("")

print("Combined DataFrame shape:", combined_df.shape)
combined_df

In [20]:
model_config = load_model_config("/Users/blairjdaniel/AI-Assistant-Springs/config/model_config.yaml")
split_params = model_config.get("split", {})
train_df, test_df = split_train_test(combined_df,
                                        test_size=split_params.get("test_size", 0.2),
                                        random_state=split_params.get("random_state", 42))
print("Train DataFrame shape:", train_df.shape)
print("Test DataFrame shape:", test_df.shape)

Train DataFrame shape: (575, 22)
Test DataFrame shape: (144, 22)
